In [1]:
import pandas as pd
import numpy as np

Load Concrete date

In [2]:
concrete_data = pd.read_csv('https://cocl.us/concrete_data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


Check the size of the data

In [3]:
concrete_data.shape

(1030, 9)

Check columns and contents

In [4]:
concrete_data.describe

<bound method NDFrame.describe of       Cement  Blast Furnace Slag  Fly Ash  Water  Superplasticizer  \
0      540.0                 0.0      0.0  162.0               2.5   
1      540.0                 0.0      0.0  162.0               2.5   
2      332.5               142.5      0.0  228.0               0.0   
3      332.5               142.5      0.0  228.0               0.0   
4      198.6               132.4      0.0  192.0               0.0   
5      266.0               114.0      0.0  228.0               0.0   
6      380.0                95.0      0.0  228.0               0.0   
7      380.0                95.0      0.0  228.0               0.0   
8      266.0               114.0      0.0  228.0               0.0   
9      475.0                 0.0      0.0  228.0               0.0   
10     198.6               132.4      0.0  192.0               0.0   
11     198.6               132.4      0.0  192.0               0.0   
12     427.5                47.5      0.0  228.0        

Let's check if we are missing values

In [5]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

Predictors are all columns exceft for Strength which is the target

In [6]:
concrete_data_cols = concrete_data.columns
predictors = concrete_data[concrete_data_cols[concrete_data_cols != 'Strength']]
target = concrete_data['Strength']

Normalize the data

In [7]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [8]:
n_cols = predictors_norm.shape[1] # number of predictors to be used to build the NN

Let's get started with Keras

In [9]:
import keras

Using TensorFlow backend.


In [10]:
from keras.models import Sequential
from keras.layers import Dense

Let's define a regression model, using 1 layer with 10 nodes, the ReLU activation function, adam as the optimizer and the mean squared error as the loss function

In [11]:
def regression_model():
    model = Sequential()
    model.add(Dense(10,activation='relu',input_shape=(n_cols,)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

Train and test the network

In [12]:
model = regression_model()
print(model.metrics_names)

Instructions for updating:
Colocations handled automatically by placer.
['loss']


Train the model using 50 epochs - loop 50 times reporting the mean squared errors

In [13]:
from sklearn.model_selection import train_test_split
scores = []
# fit the model
for i in range(50):
    X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.30, random_state=24)
    model.fit(predictors_norm, target, validation_split=0.3, epochs=50, verbose=2)
    ll = model.evaluate(X_test, y_test, verbose=0)
    scores.append(ll)
print(scores)
 

Instructions for updating:
Use tf.cast instead.
Train on 721 samples, validate on 309 samples
Epoch 1/50
 - 8s - loss: 1668.3588 - val_loss: 1184.5560
Epoch 2/50
 - 0s - loss: 1649.9504 - val_loss: 1169.8992
Epoch 3/50
 - 0s - loss: 1630.6553 - val_loss: 1155.7127
Epoch 4/50
 - 0s - loss: 1610.7405 - val_loss: 1141.0562
Epoch 5/50
 - 0s - loss: 1589.6165 - val_loss: 1126.1270
Epoch 6/50
 - 0s - loss: 1567.6805 - val_loss: 1110.7112
Epoch 7/50
 - 0s - loss: 1544.0241 - val_loss: 1094.8237
Epoch 8/50
 - 0s - loss: 1519.2676 - val_loss: 1078.5992
Epoch 9/50
 - 0s - loss: 1493.4358 - val_loss: 1061.7876
Epoch 10/50
 - 0s - loss: 1466.3304 - val_loss: 1044.5476
Epoch 11/50
 - 0s - loss: 1437.8628 - val_loss: 1026.9929
Epoch 12/50
 - 0s - loss: 1408.2095 - val_loss: 1008.6018
Epoch 13/50
 - 0s - loss: 1377.1380 - val_loss: 990.2728
Epoch 14/50
 - 0s - loss: 1345.2811 - val_loss: 970.9109
Epoch 15/50
 - 0s - loss: 1311.8191 - val_loss: 950.9198
Epoch 16/50
 - 1s - loss: 1277.1663 - val_loss: 

Report mean and standard deviation of fitting iterates

In [15]:
import statistics as st
print(scores)
print("Mean loss is %f" % st.mean(scores))
print("With standard deviation %f" % st.stdev(scores))

1292.7218805701987
